In [60]:
from datetime import datetime
import yfinance as yf

ModuleNotFoundError: No module named 'yfinance'

In [ ]:
class MyTradingStrategy:
    def __init__(self, name):
        self.__name = name

    def generate_signal(self, price_data):
        print("This methode is intended to be over ridden")
        return "Hold"

    @property
    def name(self):
        return self.__name 

In [ ]:
class MySmaTradingStrategy(MyTradingStrategy):
    def __init__(self, swindow, lwindow):
        super().__init__("MySmaTradingStrategy") 
        self.__swindow = swindow
        self.__lwindow = lwindow

    def generate_signal(self, price_data):
        if len(price_data[-self.__lwindow:]) < self.__lwindow:
            return "Hold"
        short_avg = sum(price_data[-self.__swindow:])/self.__swindow
        long_avg = sum(price_data[-self.__lwindow:])/self.__lwindow

        if short_avg > long_avg:
            return "Buy"
        elif short_avg < long_avg:
            return "Sell"
        else:
            return "Hold"

    @property
    def swindow(self):
        return self.__swindow
    
    @property
    def lwindow(self):
        return self.__lwindow

In [ ]:
class MyTrade:
    def __init__(self, strategy_name, signal, amount):
        self.__strategy_name = strategy_name
        self.__signal = signal
        self.__amount = amount
        self.__timestamp = datetime.now()

    def execute(self):
        print(f"Executing {self.__signal} trade with the strategy {self.__strategy_name} with amount {self.__amount} at the time {self.__timestamp}")

    @property
    def strategy_name(self):
        return self.__strategy_name
    
    @property
    def signal(self):
        return self.__signal
    
    @property
    def amount(self):
        return self.__amount
    
    @property
    def timestamp(self):
        return self.__timestamp

In [ ]:
class MockTradingApi:
    def __init__(self, balance):
        self.__balance = balance

    def place_order(self, trade, price):
        if trade.signal == "Buy" and self.__balance >= trade.amount * price:
            self.__balance -= trade.amount * price
            print(f"Placed a Buy trade at the {price}, Remaining balance: {self.__balance}")
        elif trade.signal == "Sell":
            self.__balance += trade.amount * price
            print(f"Placed a Sell trade at the {price}, New balance: {self.__balance}")
        else:
            print("Insufficient balance or Invalid trade signal")

        @property
        def balance(self):
            return self.__balance

In [ ]:
class MyTradingSystem:
    def __init__(self, api, strategy, symbol):
        self.__api = api
        self.__strategy = strategy
        self.__symbol = symbol

    def fetch_price_data(self):
        data = yf.download(symbol = self.__symbol, period = "1d", interval = "1m")

In [ ]:
# create strategy and compute signal
trade1 = MySmaTradingStrategy(3, 5)
strategy_name = trade1.name
signal = trade1.generate_signal([1,2,3,4,5])

# create trade and place it using the mock API
trade2 = MyTrade(strategy_name, signal, 10000)
ObjMockApi = MockTradingApi(10000)
ObjMockApi.place_order(trade2, 10000)

# optional: show execution details
trade2.execute()

Insufficient balance or Invalid trade signal
Executing Buy trade with the strategy MySmaTradingStrategy with amount 10000 at the time 2025-11-07 12:20:32.323055
